In [ ]:
import pandas as pd
import numpy as np
import re
import pickle

from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import shuffle

from nltk.stem import SnowballStemmer
from nltk import word_tokenize as nltk_wtknz

In [ ]:
data_mtv = pd.read_csv("../data/mtv_train_2.csv", sep="|")

In [ ]:
data_rt = pd.read_csv("../data/reviews_rt_all.csv", sep="|")
data_imdb = pd.read_csv("../data/imdb_small.csv", sep="|")

In [ ]:
print(data_mtv.shape)

In [ ]:
# remove rows with NaN values
data_mtv = data_mtv.dropna()

In [ ]:
print(data_mtv.shape)

In [ ]:
data_mtv.info()

In [ ]:
data_mtv.head(10)

In [ ]:
data_mtv['overall'].value_counts(normalize=True)

In [ ]:
X_train_mtv, X_test_mtv, y_train_mtv, y_test_mtv  = train_test_split(
                                                        data_mtv.reviewText, 
                                                        data_mtv.overall,
                                                        test_size=0.2, 
                                                        random_state=42)

In [ ]:
X_train_rt, X_test_rt, y_train_rt, y_test_rt  = train_test_split(
                                                        data_rt.text, 
                                                        data_rt.label,
                                                        test_size=0.2, 
                                                        random_state=42)

In [ ]:
X_train_imdb, X_test_imdb, y_train_imdb, y_test_imdb  = train_test_split(
                                                            data_imdb.text, 
                                                            data_imdb.label,
                                                            test_size=0.2, 
                                                            random_state=42)

In [ ]:
data_mix = pd.concat([data_rt, data_imdb], ignore_index=True, copy=False)
data_mix = shuffle(data_mix)

In [ ]:
X_train_mix = pd.concat([X_train_rt, X_train_imdb])
X_test_mix = pd.concat([X_test_rt, X_test_imdb])
y_train_mix = pd.concat([y_train_rt, y_train_imdb])
y_test_mix = pd.concat([y_test_rt, y_test_imdb])

In [ ]:
def tokenize(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    word_list = nltk_wtknz(text)
    stemmer = SnowballStemmer("english")
    stems = [stemmer.stem(word) for word in word_list]
    return stems

In [ ]:
pipeline = Pipeline([('vectorizer', TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 3),
                              analyzer = 'word', binary = True, max_df = 0.75)), 
                     ('classifier', LinearSVC(C=100))])

In [ ]:
model = pipeline.fit(X=X_train_mtv, y=y_train_mtv)

In [ ]:
y_pred = model.predict(X_test_mtv)

print(accuracy_score(y_test_mtv, y_pred))
print(classification_report(y_test_mtv, y_pred))

In [ ]:
y_pred = model.predict(X_test_mix)

print(accuracy_score(y_test_mix, y_pred))
print(classification_report(y_test_mix, y_pred))

In [ ]:
y_pred = model.predict(X_test_rt)

print (accuracy_score(y_test_rt, y_pred))
print(classification_report(y_test_rt, y_pred))

In [ ]:
y_pred = model.predict(X_test_imdb)

print (accuracy_score(y_test_imdb, y_pred))
print(classification_report(y_test_imdb, y_pred))

In [ ]:
with open('../dumps/m_lin_svc_mtv_out.pkl', 'wb') as f:
    pickle.dump(pipeline, f)